In [1]:
import requests # type: ignore
from urllib.parse import urljoin
import pandas as pd # type: ignore
import schedule # type: ignore
import time
from datetime import datetime, timedelta
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
import dotenv

*** Acesso a PoloTrial (API)

In [2]:
#ROTA GERAL

dotenv.load_dotenv(override=True)

session = requests.Session()

polotrial_url = os.getenv("POLOTRIAL_URL")
print(f" URL: {polotrial_url}")

payload = {
    "nome": os.getenv("POLOTRIAL_USER"),
    "password": os.getenv("POLOTRIAL_PASSWORD")
}

headers = {
    "Content-Type": "application/json"
}
session_url = f'{polotrial_url}/sessions'
print(f"session: {session_url}")
login_response = requests.request("POST", session_url, json=payload, headers=headers)

print(f"Status Code: {login_response.status_code}")

auth_cookie = login_response.cookies.get("userId")
endpoint_headers={
        "cookie": f"userId={auth_cookie}"
    }
print({endpoint_headers['cookie']})

# Mensagem de sucesso na optenção do cookie
if endpoint_headers['cookie']:
    print("Autenticação bem-sucedida. Cookie obtido.")
    print(endpoint_headers)
    print(polotrial_url)
else:
    print("Falha na autenticação. Verifique suas credenciais.")

 URL: https://api.polotrial.com/beta
session: https://api.polotrial.com/beta/sessions
Status Code: 200
{'userId=s%3AQHf9PjF0vUkyjtYe6h2UddLISs_goGHT.5%2FwFEjlzCrjNHQv%2FGZnqtwybI4GVB%2B3XyhylsNRdMbo'}
Autenticação bem-sucedida. Cookie obtido.
{'cookie': 'userId=s%3AQHf9PjF0vUkyjtYe6h2UddLISs_goGHT.5%2FwFEjlzCrjNHQv%2FGZnqtwybI4GVB%2B3XyhylsNRdMbo'}
https://api.polotrial.com/beta


In [3]:
#ROTA PROTOCOLO

protocolo_url = f"{polotrial_url}/protocolo"
protocolos_ativos = [
    "161", #Aguardando iniciação
    "39", # Recrutamento aberto
    "9236", # Recrutamento aberto
    '9263', # Aguardando Ativação do Centro
    "393", # Aguardando Ativação do Centro
    '9294', # Fase Contratual
    '819', # Aguardando o Pacote Regulatório
    '739', # Aprovação Anvisa
    '6', # Aprovação Regulatória
    '912', # Em apreciação ética
    '9163'# Aprovado pelo CEP
]
 
df_protocolos = []
 
for co_status in protocolos_ativos:
    query_params ={
        "nested":"true",
        "status_protocolo": co_status
    }
   
    try:
        response = requests.request(
            "GET",
            protocolo_url,
            headers = endpoint_headers,
            params = query_params
        )
        response.raise_for_status()
        print(f"Status {co_status} - {response.status_code}")
        protocolo_json = response.json()
        if isinstance(protocolo_json, dict):
            protocolo_json = [protocolo_json]
        df_protocolos.append(pd.DataFrame(protocolo_json))
   
    except Exception as e:
        print(f"Erro ao buscar ID {co_status}: {e}")
 
if df_protocolos:
    protocolos_dataframe = pd.concat(df_protocolos, ignore_index=True)
    print(f"DataFrame de protocolos criado com {len(protocolos_dataframe)} registros.")
else:
    protocolos_dataframe = pd.DataFrame()
    print("Nenhum dado de protocolo foi recuperado.")

Status 161 - 200
Status 39 - 200
Status 9236 - 200
Status 9263 - 200
Status 393 - 200
Status 9294 - 200
Status 819 - 200
Status 739 - 200
Status 6 - 200
Status 912 - 200
Status 9163 - 200
DataFrame de protocolos criado com 29 registros.


In [9]:
#ROTA PESSOAS

pessoas_url = f"{polotrial_url}/pessoas"
query_params = {
    "nested":"true",
}
response = requests.request("GET", pessoas_url, headers=endpoint_headers, params=query_params)

print(response.status_code)
pessoas = response.json()
pessoas_df = pd.DataFrame(pessoas)

200


In [51]:
print("SHAPE:", pessoas_df.shape)
print("COLUNAS:", pessoas_df.columns.tolist())
print(pessoas_df.head())

SHAPE: (3834, 30)
COLUNAS: ['dados_ativo', 'id', 'ds_nome', 'co_tipo_gn', 'co_equipe_gn', 'co_formacao_maior_nivel_gn', 'dt_ultimo_certificado_gcp', 'registro_profissional', 'declaracao_confidencialidade', 'custo_mensal_estimado', 'co_centro', 'equipe_padrao', 'equipe_padrao_kits', 'equipe_padrao_farmacia', 'equipe_padrao_naocega', 'filter', 'prox_vencimento_treinamento', 'co_especialidade', 'ativo', 'deleted', 'status', 'dados_co_tipo_gn', 'dados_co_equipe_gn', 'dados_co_formacao_maior_nivel_gn', 'dados_status', 'dados_centro', 'dados_equipe_padrao_farmacia', 'dados_equipe_padrao_kits', 'dados_equipe_padrao', 'dados_pessoa_especialidade']
            dados_ativo    id                 ds_nome  co_tipo_gn  \
0  {'descricao': 'Sim'}  2978                     NaN       562.0   
1  {'descricao': 'Sim'}  3545                     NaN       562.0   
2  {'descricao': 'Sim'}  1332                               649.0   
3  {'descricao': 'Sim'}  3118   Alex Gonçalves Macedo       387.0   
4  {'de

In [30]:
#ROTA AGENDA

agenda_url = f"{polotrial_url}/participante_visita"
query_params = {
    "nested":"true",
}
response = requests.request("GET", agenda_url, headers=endpoint_headers, params=query_params)

print(response.status_code)
agenda = response.json()
agenda_df = pd.DataFrame(agenda)

200


In [6]:
#ROTA PARTICIPANTES

particpantes_url = f"{polotrial_url}/participantes"
query_params = {
    "nested":"true",
}
response = requests.request("GET", particpantes_url, headers=endpoint_headers, params=query_params)

print(response.status_code)
participantes = response.json()
participantes_df = pd.DataFrame([participantes])

200


In [40]:
monitoria_url = f"{polotrial_url}/agenda"
query_params = {
    "nested":"true",
}
response = requests.request("GET", monitoria_url, headers=endpoint_headers, params=query_params)

print(response.status_code)
monitoria = response.json()
monitoria_df = pd.DataFrame(monitoria)

200


*** Visualização e modelagem para extração de GCP (cadastros ativos)

In [10]:
pessoas_df.head(5)

,dados_ativo,id,ds_nome,co_tipo_gn,co_equipe_gn,co_formacao_maior_nivel_gn,dt_ultimo_certificado_gcp,registro_profissional,declaracao_confidencialidade,custo_mensal_estimado,...,status,dados_co_tipo_gn,dados_co_equipe_gn,dados_co_formacao_maior_nivel_gn,dados_status,dados_centro,dados_equipe_padrao_farmacia,dados_equipe_padrao_kits,dados_equipe_padrao,dados_pessoa_especialidade
0,{'descricao': 'Sim'},2978,NaN,562.0,None,NaN,NaN,NaN,None,None,...,NaN,"{'id': 562, 'ds_descricao': 'Equipe padrão (ad...",None,None,None,None,None,None,None,[]
1,{'descricao': 'Sim'},3545,NaN,562.0,None,NaN,NaN,NaN,None,None,...,NaN,"{'id': 562, 'ds_descricao': 'Equipe padrão (ad...",None,None,None,None,None,None,None,[]
2,{'descricao': 'Sim'},1332,,649.0,None,NaN,NaN,NaN,None,None,...,NaN,"{'id': 649, 'ds_descricao': 'Participante'}",None,None,None,None,None,None,None,[]
3,{'descricao': 'Sim'},3118,Alex Gonçalves Macedo,387.0,None,31.0,2023-09-06,74.334/SP,None,None,...,9404.0,"{'id': 387, 'ds_descricao': 'Pesquisador Princ...",None,"{'id': 31, 'ds_descricao': 'Mestrado'}","{'id': 9404, 'ds_descricao': 'Ativo'}","{'id': 4, 'descricao': 'Santa Casa de Santos'}",None,None,None,"[{'id': 150, 'co_pessoa': 3118, 'co_pessoa_esp..."
4,{'descricao': 'Sim'},1554,Alex Vladimir Dudzic,649.0,None,NaN,NaN,NaN,None,None,...,NaN,"{'id': 649, 'ds_descricao': 'Participante'}",None,None,None,None,None,None,None,[]


In [52]:
colunas = [
    'dados_centro',
    'dt_ultimo_certificado_gcp',
    'ds_nome',
    'dados_co_tipo_gn',
    'dados_status'
]

faltantes = [c for c in colunas if c not in pessoas_df.columns]
if faltantes:
    raise Exception(f"Colunas ausentes: {faltantes}")
gcp = pessoas_df[colunas].copy()

gcp.loc[:, 'dados_centro_id'] = gcp['dados_centro'].apply(lambda x: x['id'] if x is not None else None)
gcp.loc[:, 'dados_centro_descricao'] = gcp['dados_centro'].apply(lambda x: x['descricao'] if x is not None else None)
gcp.loc[:, 'tipo_gn'] = gcp['dados_co_tipo_gn'].apply(lambda x: x['ds_descricao'] if x is not None else None)
gcp.loc[:, 'status'] = gcp['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)

gcp_ativos = gcp.drop(['dados_centro', 'dados_centro_id', 'dados_co_tipo_gn', 'dados_status'], axis=1)
gcp_modelado = gcp_ativos[gcp_ativos['status'] == 'Ativo']

gcp_modelado.head(5)

,dt_ultimo_certificado_gcp,ds_nome,dados_centro_descricao,tipo_gn,status
3,2023-09-06,Alex Gonçalves Macedo,Santa Casa de Santos,Pesquisador Principal,Ativo
5,NaN,Augusto Ferreira José,Santa Casa de Santos,Subinvestigador,Ativo
6,NaN,Dr. Ricardo Moro,Clinica de Ortopedia Campo Largo S/S LTDA,Pesquisador Principal,Ativo
312,NaN,Adão Castelo Antônio,Hospital Municipal de Barueri,Pesquisador Principal,Ativo
383,2025-06-26,Alessandro Vengjer,Santa Casa de Santos,Pesquisador Principal,Ativo


In [12]:
venc_gcp = gcp_modelado[gcp_modelado['tipo_gn'] != 'Equipe padrão (adicionada automaticamente para todos os protocolos)']
venc_gcp["Cadastro de data"] = venc_gcp["dt_ultimo_certificado_gcp"].apply(
    lambda x: "Não há data cadastrada" if pd.isna(x) else "Data cadastrada"
)

venc_gcp.rename(columns={'dt_ultimo_certificado_gcp': 'Assinatura do GCP', 'ds_nome': 'Nome', 'dados_centro_descricao': 'Centro','tipo_gn': 'Função', 'status': 'Status do Cadastro'}, inplace=True)

nova_ordem = ['Centro', 'Nome', 'Função','Status do Cadastro', 'Assinatura do GCP', "Cadastro de data"]
venc_gcp = venc_gcp[nova_ordem]
venc_gcp = venc_gcp.sort_values(by="Centro")
venc_gcp

,Centro,Nome,Função,Status do Cadastro,Assinatura do GCP,Cadastro de data
1109,A2Z Clinical Centro Avancado De Pesquisa Clini...,Dr. Fernando Henrique de Carvalho Gouvêa,Equipe Médica,Ativo,NaN,Não há data cadastrada
1276,Centro Médico Sinapse,Dr. Ricardo Moutte de Freitas,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1192,Centro Vila Olimpia,Dr. Leandro Tuzuki Cavalheiro,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1312,Centro de Oncologia do Panará,Dr. Thadeu Tiessi Suzuki,Pesquisador Principal,Ativo,2025-11-10,Data cadastrada
1108,Clin Kids,Dr. Fernando Bedoni,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
...,...,...,...,...,...,...
1559,NaN,Eduardo Ramacciotti,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1773,NaN,Francine Heiko,Subinvestigador,Ativo,NaN,Não há data cadastrada
1775,NaN,Francine Heiko Alves Perpétuo,Subinvestigador,Ativo,NaN,Não há data cadastrada
3128,NaN,Nathalia de Oliveira Westphalen,Subinvestigador,Ativo,NaN,Não há data cadastrada


In [13]:
hmcg = ['Leforte HMCG','Leforte Morumbi','Hospital Municipal de Barueri', 'Leforte Liberdade', 'Clínica CardialMed']
filtro1 = venc_gcp['Centro'].isin(hmcg)
venc_gcp_hmcg = venc_gcp[filtro1]
venc_gcp_hmcg=venc_gcp_hmcg.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_hmcg

,Centro,Nome,Função,Status do Cadastro,Assinatura do GCP,Cadastro de data
1149,Clínica CardialMed,Dr. Heron Rhydan Saad Rached,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1450,Clínica CardialMed,Dra. Maristela Gomes de Almeida,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
2015,Clínica CardialMed,Heron Rhydan Saad Rached,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
2100,Clínica CardialMed,Izabella Cordeiro Freire Saad Rached,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
312,Hospital Municipal de Barueri,Adão Castelo Antônio,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
...,...,...,...,...,...,...
1112,Leforte Morumbi,Dr. Fernando Reis Menezes,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1452,Leforte Morumbi,Dra. Mayara de Almeida Rodrigues da Costa,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1482,Leforte Morumbi,Dra. Samantha Neves,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1751,Leforte Morumbi,Fernando Reis Menezes,Pesquisador Principal,Ativo,NaN,Não há data cadastrada


In [14]:
rocio = ['Maternidade e Cirurgia Nossa Senhora do Rocio SA']
filtro2 = venc_gcp['Centro'].isin(rocio)
venc_gcp_rocio = venc_gcp[filtro2]
venc_gcp_rocio=venc_gcp_rocio.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_rocio

,Centro,Nome,Função,Status do Cadastro,Assinatura do GCP,Cadastro de data
529,Maternidade e Cirurgia Nossa Senhora do Rocio SA,Andre Luiz Santos Cerneck,Subinvestigador,Ativo,2021-09-16,Data cadastrada
531,Maternidade e Cirurgia Nossa Senhora do Rocio SA,André Ricardo Fuck,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
677,Maternidade e Cirurgia Nossa Senhora do Rocio SA,Brenno Giovanni Hernando Vidotti,Subinvestigador,Ativo,NaN,Não há data cadastrada
678,Maternidade e Cirurgia Nossa Senhora do Rocio SA,Brenno Giovanni Hernando Vidotti,Subinvestigador,Ativo,NaN,Não há data cadastrada
756,Maternidade e Cirurgia Nossa Senhora do Rocio SA,Carlos Alberto Kenji Nakashima,Subinvestigador,Ativo,NaN,Não há data cadastrada
817,Maternidade e Cirurgia Nossa Senhora do Rocio SA,César de Oliveira Lopes Dusilek,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
894,Maternidade e Cirurgia Nossa Senhora do Rocio SA,Dalton Luiz Rivabem Júnior,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
914,Maternidade e Cirurgia Nossa Senhora do Rocio SA,Daniele Nogueira Rodrigues dos Santos,Subinvestigador,Ativo,NaN,Não há data cadastrada
922,Maternidade e Cirurgia Nossa Senhora do Rocio SA,Danillo Taiguara Ramos Gomes da Silva,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1005,Maternidade e Cirurgia Nossa Senhora do Rocio SA,Dr. Alvaro Viera Moura,Subinvestigador,Ativo,NaN,Não há data cadastrada


In [15]:
iir_coord = ['Hospital das Clínicas de Itajubá', 'Saint-Beauté Clinique', 'Hospital Salvalus','Consultório Lopes e Sartorelli', 'Clínica Berger', 
            'QualiVida Higienópolis','Endolap Saúde' ]
filtro3 = venc_gcp['Centro'].isin(iir_coord)
venc_gcp_iir_coord = venc_gcp[filtro3] 
venc_gcp_iir_coord=venc_gcp_iir_coord.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_iir_coord

,Centro,Nome,Função,Status do Cadastro,Assinatura do GCP,Cadastro de data
1051,Clínica Berger,Dr. Charles Berger,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
3133,Consultório Lopes e Sartorelli,Nathalia Westphalen,Equipe Médica,Ativo,NaN,Não há data cadastrada


In [16]:
envio_viviane = ['Hospital Pilar','Santa casa de São Paulo','Hospital São Francisco de Ribeirão Preto',
                'Hospital Antônio Prudente','Clínica CardialMed','CLINAR - Clínica de Aparelhos RespiratÃ³rios',
                'Santa Casa de Fortaleza','Hospital Vera Cruz','Hapvida','Unimed Brusque',
                'Hospital São José das Doenças Infecciosas','Otorhinus Clínica Médica','Hospital São Francisco de Araraquara',
                'Maternidade Octaviano Neves','Clínica Infectologie','Hospital Teresa de Lisieux',
                'Hospital RioMar de Belém','Hospital e Maternidade Eugênia Pinheiro',
                'Hospital São Francisco Saúde','Hospital do Coração de Campinas','Aliança Cavernoma Brasil',
                'Faculdade de Medicina de Ribeirão Preto - USP ','Centro Clínico Zona Sul ']
filtro4 = venc_gcp['Centro'].isin(envio_viviane)
venc_gcp_envio_viviane = venc_gcp[filtro4]
venc_gcp_envio_viviane=venc_gcp_envio_viviane.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_envio_viviane

,Centro,Nome,Função,Status do Cadastro,Assinatura do GCP,Cadastro de data
1149,Clínica CardialMed,Dr. Heron Rhydan Saad Rached,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1450,Clínica CardialMed,Dra. Maristela Gomes de Almeida,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
2015,Clínica CardialMed,Heron Rhydan Saad Rached,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
2100,Clínica CardialMed,Izabella Cordeiro Freire Saad Rached,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1743,Faculdade de Medicina de Ribeirão Preto - USP,Fernando Belissimo Rodrigues,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
761,Hospital Vera Cruz,Carlos Ernesto Ferreira Starling,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1257,Otorhinus Clínica Médica,Dr. Rafael Leonardo Emerich Lentz Martins,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
3289,Otorhinus Clínica Médica,Rafael Leonardo Emerich Lentz Martins,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1017,Unimed Brusque,Dr. Antonio Carlos de Mattos Roxo,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
1063,Unimed Brusque,Dr. Daniel Rodrigo Klein,Pesquisador Principal,Ativo,NaN,Não há data cadastrada


In [17]:
stacasa_santos = ['Santa Casa de Santos']
filtro5 = venc_gcp['Centro'].isin(stacasa_santos)
venc_gcp_stacasa_santos = venc_gcp[filtro5]
venc_gcp_stacasa_santos=venc_gcp_stacasa_santos.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_stacasa_santos

,Centro,Nome,Função,Status do Cadastro,Assinatura do GCP,Cadastro de data
3,Santa Casa de Santos,Alex Gonçalves Macedo,Pesquisador Principal,Ativo,2023-09-06,Data cadastrada
5,Santa Casa de Santos,Augusto Ferreira José,Subinvestigador,Ativo,NaN,Não há data cadastrada
383,Santa Casa de Santos,Alessandro Vengjer,Pesquisador Principal,Ativo,2025-06-26,Data cadastrada
516,Santa Casa de Santos,André Galante Alencar Aranha,Pesquisador Principal,Ativo,2021-09-23,Data cadastrada
518,Santa Casa de Santos,André Gustavo Pereira Magalhães,Subinvestigador,Ativo,NaN,Não há data cadastrada
...,...,...,...,...,...,...
3392,Santa Casa de Santos,Roberto Higa Junior,Subinvestigador,Ativo,NaN,Não há data cadastrada
3479,Santa Casa de Santos,Samuel Brunini Petrarolha,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
3478,Santa Casa de Santos,Samuel Brunini Petrarolha,Pesquisador Principal,Ativo,NaN,Não há data cadastrada
3771,Santa Casa de Santos,Vitor Hugo Straub,Pesquisador Principal,Ativo,NaN,Não há data cadastrada


In [18]:
capibaribe = ['Hospital do Capibaribe']
filtro6 = venc_gcp['Centro'].isin(capibaribe)
venc_gcp_capibaribe = venc_gcp[filtro6]
venc_gcp_capibaribe=venc_gcp_capibaribe.sort_values(by=['Centro', 'Nome'], ascending= [True, True])
venc_gcp_capibaribe

,Centro,Nome,Função,Status do Cadastro,Assinatura do GCP,Cadastro de data
1163,Hospital do Capibaribe,Dr. Jayme José Gouveia Filho,Equipe Médica,Ativo,2024-09-23,Data cadastrada
1447,Hospital do Capibaribe,Dra. Mariane Teodoro Fernandes Rodrigues,Pesquisador Principal,Ativo,2024-06-14,Data cadastrada
2800,Hospital do Capibaribe,Mariane Teodoro Fernandes Rodrigues,Pesquisador Principal,Ativo,NaN,Não há data cadastrada


***Função para cálculo da data de vencimento de GCP.

In [19]:
def verificar_vencimento_contratos(df_contratos, nome_centro, dias_para_vencimento=30):

    df_contratos['Assinatura do GCP'] = pd.to_datetime(df_contratos['Assinatura do GCP'], errors='coerce')

    hoje = datetime.today()
    limite_vencimento = hoje + timedelta(days=dias_para_vencimento)

    df_contratos['data_vencimento'] = df_contratos['Assinatura do GCP'] + timedelta(days=2 * 365)

    filtro_vencendo = (df_contratos['data_vencimento'] > hoje) & (df_contratos['data_vencimento'] <= limite_vencimento)

    filtro_sem_data = df_contratos['Assinatura do GCP'].isna()

    contratos_relevantes = df_contratos[filtro_vencendo | filtro_sem_data]

    if contratos_relevantes.empty:
        print(f"🔹 Não há contratos GCP sem data registrada no sistema ou com vencimento nos próximos {dias_para_vencimento} dias para {nome_centro}.")
    else:
        print(f"⚠️{nome_centro} - Os profissionais abaixo listados não tem data de GCP na PoloTrial ou a data informada irá vencer nos próximos {dias_para_vencimento} dias:")
        print(contratos_relevantes[['Centro', 'Nome', 'Função','Status do Cadastro', 'Assinatura do GCP', "Cadastro de data"]])

    return contratos_relevantes

In [ ]:
contratos_hmcg = verificar_vencimento_contratos(venc_gcp_hmcg, "HMCG", dias_para_vencimento=30)
contratos_rocio = verificar_vencimento_contratos(venc_gcp_rocio, "Rocío", dias_para_vencimento=30)
contratos_iir_coord = verificar_vencimento_contratos(venc_gcp_iir_coord, "IRR COORD", dias_para_vencimento=30)
contratos_vivi = verificar_vencimento_contratos(venc_gcp_envio_viviane, "Envio Viviane", dias_para_vencimento=30)
contratos_stacsantos = verificar_vencimento_contratos(venc_gcp_stacasa_santos, "Santa Casa de Santos", dias_para_vencimento=30)
contratos_capibaribe = verificar_vencimento_contratos(venc_gcp_capibaribe, "Capibaribe", dias_para_vencimento=30)


⚠️HMCG - Os profissionais abaixo listados não tem data de GCP na PoloTrial ou a data informada irá vencer nos próximos 30 dias:
                             Centro  \
1149             Clínica CardialMed   
1450             Clínica CardialMed   
2015             Clínica CardialMed   
2100             Clínica CardialMed   
312   Hospital Municipal de Barueri   
...                             ...   
1112                Leforte Morumbi   
1452                Leforte Morumbi   
1482                Leforte Morumbi   
1751                Leforte Morumbi   
2884                Leforte Morumbi   

                                           Nome                 Função  \
1149               Dr. Heron Rhydan Saad Rached  Pesquisador Principal   
1450            Dra. Maristela Gomes de Almeida  Pesquisador Principal   
2015                   Heron Rhydan Saad Rached  Pesquisador Principal   
2100       Izabella Cordeiro Freire Saad Rached  Pesquisador Principal   
312                        Adão C

***Função para envio dos emails de GCP

In [21]:
css_hover = """
<style>
/* Reset básico */
table {
    border-collapse: collapse;
    width: 100%;
    font-family: Arial, Helvetica, sans-serif;
    font-size: 13px;
}

/* Cabeçalho */
th {
    background-color: #007bff;
    color: #ffffff;
    border: 1px solid #d0d0d0;
    padding: 8px;
    text-align: left;
    font-weight: bold;
}

/* Células */
td {
    border: 1px solid #d0d0d0;
    padding: 8px;
    text-align: left;
    vertical-align: middle;
    color: #000000;
}

/* Linhas alternadas (mais compatível que hover) */
tr:nth-child(even) {
    background-color: #f8f9fa;
}

/* Hover (funciona em Gmail/Web, ignorado no Outlook) */
tr:hover {
    background-color: #fff3cd;
}

/* Responsividade simples */
@media screen and (max-width: 600px) {
    table {
        font-size: 12px;
    }
}
</style>
"""

Variáveis de envio de email

In [ ]:
smtp_server = os.getenv("EMAIL_SERVER")
email_port = int(os.getenv("EMAIL_PORT"))
email_usuario = os.getenv("EMAIL_USERNAME")
email_senha = os.getenv("EMAIL_PASSWORD")
enviar_para = os.getenv('ENVIAR_PARA')
destinatario_hmcg = os.getenv('DESTINATARIO_HMCG')
destinatario_rocio =  os.getenv('DESTINATARIO_ROCIO')
destinatario_scs = os.getenv('DESTINATARIO_SCS')
destinatario_iir =  os.getenv('DESTINATARIO_IIR')
destinatario_vivi =  os.getenv('DESTINATARIO_VIVIANE')
destinatario_capibaribe =  os.getenv('DESTINATARIO_CAPIBARIBE')

In [23]:
def enviar_email(destinatarios, df_tratado, nome_grupo):
    if df_tratado is None or df_tratado.empty:
        print("Não há contratos vencidos para enviar por e-mail.")
        return

    try:
        # Garante lista de destinatários
        if isinstance(destinatarios, str):
            destinatarios = [destinatarios]

        # Tabela HTML
        colunas = ['Centro', 'Nome', 'Função','Status do Cadastro', 'Assinatura do GCP', "Cadastro de data"]
        tabela_html = df_tratado[colunas].to_html(
            index=False,
            escape=False,
            border=0,
            justify="left",
            classes="table"
        )

        # Configuração do e-mail
        msg = MIMEMultipart("alternative")
        msg["From"] = email_usuario
        msg["To"] = email_usuario
        msg["Bcc"] = ", ".join(destinatarios)
        msg["Subject"] = f"[{nome_grupo}] GCP sem data registrada ou com vencimento próximo"

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <h2>Vencimento de GCP - {nome_grupo}</h2>

                <p>Bom dia,</p>

                <p>
                    Segue abaixo relação de GCPs que não possuem data registradano sistema ou que apresentam vencimento próximo:
                </p>
                {tabela_html}
                <br>
                <p>
                    Este e-mail é gerado automaticamente a partir de informações inseridas na <strong>Polo Trial</strong>.
                </p>
                <p>
                    Qualquer dúvida, por favor, contate o <strong><u>time BI - SVRI</u></strong>.
                </p>
            </body>
        </html>
        """

        msg.attach(MIMEText(body, "html"))

        # Envio
        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            server.send_message(msg)

        print(f"E-mail enviado com sucesso para o grupo: {nome_grupo}")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")


In [ ]:
enviar_email([destinatario_hmcg], contratos_hmcg, "HMCG")
enviar_email([destinatario_rocio], contratos_rocio, "Rocio")
enviar_email([destinatario_iir], contratos_iir_coord, "IIR")
enviar_email([destinatario_scs], contratos_stacsantos, "Sta. Santos")
enviar_email([destinatario_vivi], contratos_vivi, "Viviane")
enviar_email([destinatario_capibaribe], contratos_capibaribe, "Capibaribe")

E-mail enviado com sucesso para o grupo: HMCG
E-mail enviado com sucesso para o grupo: Rocio
E-mail enviado com sucesso para o grupo: IIR
E-mail enviado com sucesso para o grupo: Sta. Santos
E-mail enviado com sucesso para o grupo: Viviane
E-mail enviado com sucesso para o grupo: Capiberibe


***Modelagem para envio de Visitas de Seguimento

In [25]:
protocolo = pd.DataFrame(protocolos_dataframe)
protocolo.head(5)

,id,titulo_protocolo,cor_agenda,numero_protocolo,apelido_protocolo,coordenador,pi,pesquisador_backup,co_pessoa_regulatorio,data_cadastro,...,dados_medicamento_do_estudo,dados_pendencias_regulatorio,dados_status_feasibility,dados_status_contrato,dados_status_orcamento,dados_status_regulatorio,dados_status_recrutamento,dados_status_financeiro,dados_assistente_de_pesquisa,dados_dificuldade_recrutamento
0,784,"Um estudo randomizado, duplo-cego, controlado ...",#000000,LX4211.1-314-HCM,SONATA-HCM,2631.0,271,NaN,2631.0,2023-07-17T10:14:16.000Z,...,None,"{'id': 40, 'ds_descricao': 'Sim'}","{'id': 824, 'ds_descricao': 'Respondido'}","{'id': 9, 'ds_descricao': 'Em negociação com o...",None,"{'id': 914, 'ds_descricao': 'Aprovado'}",None,None,None,"{'id': 9292, 'ds_descricao': 'Moderado'}"
1,1256,"Estudo VISIONAIRE (Vitamin K AntagonISt, Facto...",#000000,,VISIONAIRE,2829.0,3050,NaN,2631.0,2023-11-30T09:28:12.000Z,...,None,"{'id': 40, 'ds_descricao': 'Sim'}","{'id': 824, 'ds_descricao': 'Respondido'}","{'id': 908, 'ds_descricao': 'Assinado'}","{'id': 877, 'ds_descricao': 'Aprovado'}","{'id': 918, 'ds_descricao': 'Em análise do CEP'}",None,None,None,None
2,1661,"Um ensaio randomizado, controlado por placebo,...",#9a514c,BIOTEST-996,EsSCAPE-996,239.0,250,NaN,2807.0,2024-06-07T14:43:21.000Z,...,None,"{'id': 40, 'ds_descricao': 'Sim'}","{'id': 824, 'ds_descricao': 'Respondido'}",None,None,"{'id': 914, 'ds_descricao': 'Aprovado'}","{'id': 936, 'ds_descricao': 'Recrutamento aber...",None,None,"{'id': 9292, 'ds_descricao': 'Moderado'}"
3,2229,"Estudo de fase II, multicêntrico, duplo-cego, ...",#000000,CS45570,CS45570,4528.0,319,NaN,2631.0,2024-11-26T11:25:36.000Z,...,None,"{'id': 40, 'ds_descricao': 'Sim'}","{'id': 824, 'ds_descricao': 'Respondido'}","{'id': 908, 'ds_descricao': 'Assinado'}",None,"{'id': 914, 'ds_descricao': 'Aprovado'}","{'id': 928, 'ds_descricao': 'Aguardando ativaç...",None,None,"{'id': 9291, 'ds_descricao': 'Difícil'}"
4,2372,"Estudo randomizado, multicêntrico, duplo cego,...",#000000,,Oxandrolona,2622.0,4166,3662.0,2631.0,2025-03-17T15:54:16.000Z,...,None,"{'id': 41, 'ds_descricao': 'Não'}","{'id': 824, 'ds_descricao': 'Respondido'}","{'id': 908, 'ds_descricao': 'Assinado'}",None,"{'id': 914, 'ds_descricao': 'Aprovado'}",None,None,None,"{'id': 9292, 'ds_descricao': 'Moderado'}"


In [26]:
centros = protocolo[['id','apelido_protocolo', 'numero_protocolo','co_externo','apelido_centro']].copy()
centros.head(5)

,id,apelido_protocolo,numero_protocolo,co_externo,apelido_centro
0,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed
1,1256,VISIONAIRE,,None,VISIONAIRE - SVRI Clínica Morumbi
2,1661,EsSCAPE-996,BIOTEST-996,None,EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...
3,2229,CS45570,CS45570,None,CS45570 - Leforte HMCG
4,2372,Oxandrolona,,None,Oxandrolona - Santa Casa de Santos


In [32]:
seguimento = agenda_df[['dados_participante', 'data_estimada', 'dados_status', 'dados_visita']].copy()

seguimento.loc[:, 'dados_participante_id'] = seguimento['dados_participante'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'id_participante'] = seguimento['dados_participante'].apply(lambda x: x['id_participante'] if x is not None else None)
seguimento.loc[:, 'dados_protocolo'] = seguimento['dados_participante'].apply(lambda x: x['dados_protocolo'] if x is not None else None)

seguimento.loc[:, 'dados_status_id'] = seguimento['dados_status'].apply(lambda x: x['id'] if x is not None else None)
seguimento.loc[:, 'ds_descricao'] = seguimento['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)
seguimento.loc[:, 'ds_nome_visita'] = seguimento['dados_visita'].apply(lambda x: x['ds_nome_visita'] if x is not None else None)

seguimento.loc[:, 'apelido_protocolo'] = seguimento['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)
seguimento['data_estimada'] = pd.to_datetime(seguimento['data_estimada'], errors='coerce')
seguimento.head(5)

,dados_participante,data_estimada,dados_status,dados_visita,dados_participante_id,id_participante,dados_protocolo,dados_status_id,ds_descricao,ds_nome_visita,apelido_protocolo
0,"{'id': 54, 'co_protocolo': 4, 'id_participante...",2021-01-05 00:00:00+00:00,"{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",54,1001,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",20,Realizada,Triagem,BTK
1,"{'id': 56, 'co_protocolo': 4, 'id_participante...",2021-01-07 00:00:00+00:00,"{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",56,1002,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",20,Realizada,Triagem,BTK
2,"{'id': 57, 'co_protocolo': 4, 'id_participante...",2021-01-08 00:00:00+00:00,"{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",57,1003,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",20,Realizada,Triagem,BTK
3,"{'id': 58, 'co_protocolo': 4, 'id_participante...",2021-01-12 00:00:00+00:00,"{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",58,1004,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",20,Realizada,Triagem,BTK
4,"{'id': 59, 'co_protocolo': 4, 'id_participante...",2021-01-14 00:00:00+00:00,"{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",59,1005,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",20,Realizada,Triagem,BTK


In [33]:
seguimento_tratado = seguimento.drop(['dados_participante','dados_status','id_participante','dados_status_id'], axis=1)
seguimento_tratado.loc[:, 'id'] = seguimento['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
seguimento_tratado.head(5)

,data_estimada,dados_visita,dados_participante_id,dados_protocolo,ds_descricao,ds_nome_visita,apelido_protocolo,id
0,2021-01-05 00:00:00+00:00,"{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",54,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",Realizada,Triagem,BTK,4
1,2021-01-07 00:00:00+00:00,"{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",56,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",Realizada,Triagem,BTK,4
2,2021-01-08 00:00:00+00:00,"{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",57,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",Realizada,Triagem,BTK,4
3,2021-01-12 00:00:00+00:00,"{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",58,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",Realizada,Triagem,BTK,4
4,2021-01-14 00:00:00+00:00,"{'id': 1, 'ds_nome_visita': 'Triagem ', 'co_ex...",59,"{'id': 4, 'apelido_protocolo': 'BTK', 'co_coor...",Realizada,Triagem,BTK,4


In [34]:
seguimentos = pd.merge(centros, seguimento_tratado, on='id', how='inner')
seguimentos

,id,apelido_protocolo_x,numero_protocolo,co_externo,apelido_centro,data_estimada,dados_visita,dados_participante_id,dados_protocolo,ds_descricao,ds_nome_visita,apelido_protocolo_y
0,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,2026-01-16 00:00:00+00:00,"{'id': 3405, 'ds_nome_visita': 'Visita 1 - Tri...",3286,"{'id': 784, 'apelido_protocolo': 'SONATA-HCM',...",Realizada,Visita 1 - Triagem,SONATA-HCM
1,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,2026-01-16 00:00:00+00:00,"{'id': 3405, 'ds_nome_visita': 'Visita 1 - Tri...",3287,"{'id': 784, 'apelido_protocolo': 'SONATA-HCM',...",Realizada,Visita 1 - Triagem,SONATA-HCM
2,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,NaT,"{'id': 3412, 'ds_nome_visita': 'Visita de ence...",3287,"{'id': 784, 'apelido_protocolo': 'SONATA-HCM',...",Pendente,Visita de encerramento precoce,SONATA-HCM
3,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,NaT,"{'id': 3414, 'ds_nome_visita': 'Unscheduled', ...",3287,"{'id': 784, 'apelido_protocolo': 'SONATA-HCM',...",Pendente,Unscheduled,SONATA-HCM
4,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,2026-02-06 00:00:00+00:00,"{'id': 3406, 'ds_nome_visita': 'Visita 2 - Ran...",3287,"{'id': 784, 'apelido_protocolo': 'SONATA-HCM',...",Pendente,Visita 2 - Randomização,SONATA-HCM
...,...,...,...,...,...,...,...,...,...,...,...,...
895,1531,DLG,,NaN,DLG - Santa Casa de Santos,2025-09-30 00:00:00+00:00,"{'id': 2996, 'ds_nome_visita': 'V4 - Dia 42', ...",3192,"{'id': 1531, 'apelido_protocolo': 'DLG', 'co_c...",Realizada,V4 - Dia 42,DLG
896,1531,DLG,,NaN,DLG - Santa Casa de Santos,2025-10-07 00:00:00+00:00,"{'id': 2997, 'ds_nome_visita': 'CT4 - Dia 49',...",3192,"{'id': 1531, 'apelido_protocolo': 'DLG', 'co_c...",Realizada,CT4 - Dia 49,DLG
897,1531,DLG,,NaN,DLG - Santa Casa de Santos,2025-10-14 00:00:00+00:00,"{'id': 2998, 'ds_nome_visita': 'V5 - Dia 56', ...",3192,"{'id': 1531, 'apelido_protocolo': 'DLG', 'co_c...",Realizada,V5 - Dia 56,DLG
898,1531,DLG,,NaN,DLG - Santa Casa de Santos,2025-10-21 00:00:00+00:00,"{'id': 2999, 'ds_nome_visita': 'Visita de Desc...",3192,"{'id': 1531, 'apelido_protocolo': 'DLG', 'co_c...",Realizada,Visita de Descontinuação VD (Caso o paciente s...,DLG


In [35]:
seguimentos['apelido_protocolo_x'] = seguimentos['apelido_protocolo_x'].str.strip()
visitas_filtrado = seguimentos[seguimentos['ds_descricao'].str.contains('Pendente')]
nova_ordem = ["apelido_centro", "dados_participante_id", "ds_nome_visita","data_estimada","ds_descricao"]
visitas_reordenado = visitas_filtrado[nova_ordem].rename(
    columns={
        'apelido_centro':'Estudo/Centro',
        'dados_participante_id': 'ID Participante',
        'ds_nome_visita': 'Tipo Visita',
        'data_estimada':'Data Estimada',
        'ds_descricao':'Status'
    }
)
visitas_reordenado['Data Estimada'] = pd.to_datetime(visitas_reordenado['Data Estimada'], errors='coerce')
visitas_reordenado

,Estudo/Centro,ID Participante,Tipo Visita,Data Estimada,Status
2,SONATA-HCM - Clínica CardialMed,3287,Visita de encerramento precoce,NaT,Pendente
3,SONATA-HCM - Clínica CardialMed,3287,Unscheduled,NaT,Pendente
4,SONATA-HCM - Clínica CardialMed,3287,Visita 2 - Randomização,2026-02-06 00:00:00+00:00,Pendente
5,SONATA-HCM - Clínica CardialMed,3287,Visita 3 - Semana 2 (Telefônica),2026-02-20 00:00:00+00:00,Pendente
6,SONATA-HCM - Clínica CardialMed,3287,Visita 4 - Semana 4,2026-03-06 00:00:00+00:00,Pendente
...,...,...,...,...,...
865,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,3139,Visita 16,2028-06-01 00:00:00+00:00,Pendente
866,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,3139,FU,NaT,Pendente
867,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,3139,Visita Não Programada,NaT,Pendente
868,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,3139,EOS,NaT,Pendente


In [36]:
datas_visitas = visitas_reordenado.copy()
datas_visitas['Data Estimada'] = pd.to_datetime(datas_visitas['Data Estimada'], errors='coerce')
datas_visitas = visitas_reordenado.dropna(subset=["Data Estimada"])
datas_visitas = datas_visitas.sort_values(by= "Data Estimada", ascending=True)
datas_visitas = datas_visitas[~datas_visitas["Tipo Visita"].isin(["Unscheduled", "Triagem", "End of Study"])]
valores_contagem = datas_visitas['Tipo Visita'].value_counts()
datas_visitas

,Estudo/Centro,ID Participante,Tipo Visita,Data Estimada,Status
564,FREXALT - Leforte HMCG,3296,V2 - D1 - Randomização,2026-01-28 00:00:00+00:00,Pendente
557,FREXALT - Leforte HMCG,3291,V2 - D1 - Randomização,2026-02-04 00:00:00+00:00,Pendente
535,FREXALT - Leforte HMCG,3138,Semana 28,2026-02-04 00:00:00+00:00,Pendente
514,FREXALT - Leforte HMCG,3133,Semana 28,2026-02-04 00:00:00+00:00,Pendente
94,CS45570 - Leforte HMCG,3273,Semana 6,2026-02-04 00:00:00+00:00,Pendente
...,...,...,...,...,...
771,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,3265,Visita 16,2028-09-04 00:00:00+00:00,Pendente
801,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,3269,Visita 16,2028-09-26 00:00:00+00:00,Pendente
815,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,3268,Visita 16,2028-09-27 00:00:00+00:00,Pendente
829,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,3270,Visita 16,2028-10-02 00:00:00+00:00,Pendente


In [37]:
def verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20):
    df = datas_visitas.copy()

    df['Data Estimada'] = (
        pd.to_datetime(df['Data Estimada'], errors='coerce')
        .dt.tz_localize(None)   # 👈 REMOVE UTC
    )

    hoje = pd.Timestamp.today().normalize()
    limite_visita = hoje + pd.Timedelta(days=dias_para_visita)

    visitas_futuras = df[
        (df['Data Estimada'] >= hoje) &
        (df['Data Estimada'] <= limite_visita)
    ]

    return visitas_futuras

visitas_20_dias = verificar_visitas_proximos_dias(datas_visitas, dias_para_visita=20)

In [38]:
def enviar_emails(próximas_visitas):
    global enviar_para  # Acessa a variável global 'enviar_para'
    
    try:
        if próximas_visitas is None or próximas_visitas.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        colunas_esperadas = {'Estudo/Centro', 'ID Participante', 'Tipo Visita', 'Data Estimada', 'Status'}
        if not colunas_esperadas.issubset(próximas_visitas.columns):
            print("Erro: DataFrame não contém todas as colunas esperadas.")
            return

        if not all([smtp_server, email_usuario, email_senha, email_port]):
            print("Erro: Configurações de e-mail estão incompletas.")
            return

        # Validação de e-mails com regex
        email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
        if isinstance(enviar_para, list):
            enviar_para = [email.strip() for email in enviar_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            enviar_para = []

        # Criação da tabela HTML
        tabela_html = próximas_visitas[list(colunas_esperadas)].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        # Montagem da mensagem
        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Seguimento - próximos 20 dias"
        
        # Define o campo BCC apenas se houver destinatários válidos
        if enviar_para:
            msg['Bcc'] = ', '.join(enviar_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de seguimento programadas para os próximos 20 dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        # Envio de e-mail
        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            
            # Lista final de destinatários
            destinatarios = [email_usuario] + enviar_para
            if destinatarios:
                server.sendmail(email_usuario, destinatarios, msg.as_string())
                print("E-mail de visitas pendentes enviado com sucesso!")
            else:
                print("Nenhum destinatário válido. E-mail não enviado.")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(visitas_20_dias)

E-mail de visitas pendentes enviado com sucesso!


**** Visitas de Monitoria

In [41]:
monitoria_df

,id,tipo_evento,titulo,ds_co_industria,ds_co_hc,co_participante,tipo,status_evento,data_estimada,data_estimada_fim,...,co_externo,data_limite_min,data_limite_max,data_ideal,dados_tipo,dados_status,dados_protocolo,dados_participante,dados_responsavel,dados_local
0,1,1,Centriguga,,,,4,20,2022-04-28T00:00:00.000Z,NaN,...,None,NaN,NaN,NaN,"{'id': 4, 'tipo': 'Calibração'}","{'id': 20, 'ds_descricao': 'Realizada'}",None,None,None,None
1,4,1,contrato V6,,,,6,20,2022-07-06T00:00:00.000Z,NaN,...,None,NaN,NaN,NaN,"{'id': 6, 'tipo': 'Contrato'}","{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 53, 'apelido_protocolo': 'Dipro', 'co_c...",None,None,None
2,5,1,contrato_ Graviti,,,,6,20,2022-12-29T00:00:00.000Z,NaN,...,None,NaN,NaN,NaN,"{'id': 6, 'tipo': 'Contrato'}","{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 322, 'apelido_protocolo': 'Graviti', 'c...",None,None,None
3,6,1,contrato_ Graviti,,,,6,20,2022-12-29T00:00:00.000Z,NaN,...,None,NaN,NaN,NaN,"{'id': 6, 'tipo': 'Contrato'}","{'id': 20, 'ds_descricao': 'Realizada'}","{'id': 322, 'apelido_protocolo': 'Graviti', 'c...",None,None,None
4,7,1,Vencimento GCP,,,,7,20,2025-08-16T00:00:00.000Z,2025-08-16T00:00:00.000Z,...,None,NaN,NaN,NaN,"{'id': 7, 'tipo': 'Vencimento GCP'}","{'id': 20, 'ds_descricao': 'Realizada'}",None,None,"{'id': 239, 'ds_nome': 'Caroline Pinheiro Vian...","{'id': 765, 'ds_descricao': 'Virtual'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17170,40190,2,Visita 16,1076012009,1076012,3139,1,18,2028-06-01T00:00:00.000Z,NaN,...,None,2028-05-11T00:00:00.000Z,2028-06-22T00:00:00.000Z,2028-06-01T00:00:00.000Z,"{'id': 1, 'tipo': 'Visita'}","{'id': 18, 'ds_descricao': 'Pendente'}","{'id': 655, 'apelido_protocolo': 'EASi-HF - 13...","{'id': 3139, 'id_participante': '1076012009', ...",None,None
17171,40192,2,FU,1076012009,1076012,3139,1,18,NaN,NaN,...,None,NaN,NaN,NaN,"{'id': 1, 'tipo': 'Visita'}","{'id': 18, 'ds_descricao': 'Pendente'}","{'id': 655, 'apelido_protocolo': 'EASi-HF - 13...","{'id': 3139, 'id_participante': '1076012009', ...",None,None
17172,40193,2,Visita Não Programada,1076012009,1076012,3139,1,18,NaN,NaN,...,None,NaN,NaN,NaN,"{'id': 1, 'tipo': 'Visita'}","{'id': 18, 'ds_descricao': 'Pendente'}","{'id': 655, 'apelido_protocolo': 'EASi-HF - 13...","{'id': 3139, 'id_participante': '1076012009', ...",None,None
17173,40194,2,EOS,1076012009,1076012,3139,1,18,NaN,NaN,...,None,NaN,NaN,NaN,"{'id': 1, 'tipo': 'Visita'}","{'id': 18, 'ds_descricao': 'Pendente'}","{'id': 655, 'apelido_protocolo': 'EASi-HF - 13...","{'id': 3139, 'id_participante': '1076012009', ...",None,None


In [42]:
monitoria = monitoria_df[['dados_protocolo', 'dados_tipo', 'dados_status', 'data_estimada_filter']].copy()

monitoria.loc[:, 'dados_protocolo_id'] = monitoria['dados_protocolo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'apelido_protocolo'] = monitoria['dados_protocolo'].apply(lambda x: x['apelido_protocolo'] if x is not None else None)

monitoria.loc[:, 'dados_tipo_id'] = monitoria['dados_tipo'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'dados_tipo_tipo'] = monitoria['dados_tipo'].apply(lambda x: x['tipo'] if x is not None else None)

monitoria.loc[:, 'dados_status_id'] = monitoria['dados_status'].apply(lambda x: x['id'] if x is not None else None)
monitoria.loc[:, 'ds_descricao'] = monitoria['dados_status'].apply(lambda x: x['ds_descricao'] if x is not None else None)

monitoria = monitoria.drop(['dados_protocolo','dados_status', 'dados_tipo_id','dados_status_id'], axis=1)
monitoria.rename(columns={'dados_protocolo_id': 'id'}, inplace=True)

monitoria.head(2)

,dados_tipo,data_estimada_filter,id,apelido_protocolo,dados_tipo_tipo,ds_descricao
0,"{'id': 4, 'tipo': 'Calibração'}",2022-04-28,NaN,NaN,Calibração,Realizada
1,"{'id': 6, 'tipo': 'Contrato'}",2022-07-06,53.0,Dipro,Contrato,Realizada


In [43]:
monitorias = pd.merge(centros, monitoria, on='id', how='inner')
monitorias

,id,apelido_protocolo_x,numero_protocolo,co_externo,apelido_centro,dados_tipo,data_estimada_filter,apelido_protocolo_y,dados_tipo_tipo,ds_descricao
0,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,"{'id': 2, 'tipo': 'Monitoria'}",2026-02-13,SONATA-HCM,Monitoria,Pendente
1,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,"{'id': 1, 'tipo': 'Visita'}",2026-01-16,SONATA-HCM,Visita,Realizada
2,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,"{'id': 1, 'tipo': 'Visita'}",2026-01-16,SONATA-HCM,Visita,Realizada
3,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,"{'id': 1, 'tipo': 'Visita'}",NaN,SONATA-HCM,Visita,Pendente
4,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,"{'id': 1, 'tipo': 'Visita'}",NaN,SONATA-HCM,Visita,Pendente
...,...,...,...,...,...,...,...,...,...,...
973,1531,DLG,,NaN,DLG - Santa Casa de Santos,"{'id': 1, 'tipo': 'Visita'}",2025-10-14,DLG,Visita,Realizada
974,1531,DLG,,NaN,DLG - Santa Casa de Santos,"{'id': 1, 'tipo': 'Visita'}",2025-10-21,DLG,Visita,Realizada
975,1531,DLG,,NaN,DLG - Santa Casa de Santos,"{'id': 1, 'tipo': 'Visita'}",NaN,DLG,Visita,Não realizada
976,2230,EASi-HF reduzido - 1378-0018,1378-0018,NaN,EASi-HF reduzido - 1378-0018 - Maternidade e C...,"{'id': 2, 'tipo': 'Monitoria'}",2025-10-23,EASi-HF reduzido - 1378-0018,Monitoria,Realizada


In [44]:
monitorias_filtrado = monitorias[monitorias['dados_tipo_tipo'].str.contains('Monitoria')]
monitorias_filtrado['apelido_protocolo_x'] = monitorias_filtrado['apelido_protocolo_x'].str.strip()
monitorias_filtrado

,id,apelido_protocolo_x,numero_protocolo,co_externo,apelido_centro,dados_tipo,data_estimada_filter,apelido_protocolo_y,dados_tipo_tipo,ds_descricao
0,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,"{'id': 2, 'tipo': 'Monitoria'}",2026-02-13,SONATA-HCM,Monitoria,Pendente
20,1661,EsSCAPE-996,BIOTEST-996,None,EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...,"{'id': 2, 'tipo': 'Monitoria'}",2024-07-03,EsSCAPE-996,Monitoria,Realizada
21,1661,EsSCAPE-996,BIOTEST-996,None,EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...,"{'id': 2, 'tipo': 'Monitoria'}",2025-07-24,EsSCAPE-996,Monitoria,Realizada
22,1661,EsSCAPE-996,BIOTEST-996,None,EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...,"{'id': 2, 'tipo': 'Monitoria'}",2025-09-16,EsSCAPE-996,Monitoria,Realizada
23,1661,EsSCAPE-996,BIOTEST-996,None,EsSCAPE-996 - Maternidade e Cirurgia Nossa Sen...,"{'id': 2, 'tipo': 'Monitoria'}",2025-09-25,EsSCAPE-996,Monitoria,Realizada
...,...,...,...,...,...,...,...,...,...,...
943,1531,DLG,,NaN,DLG - Santa Casa de Santos,"{'id': 2, 'tipo': 'Monitoria'}",2025-05-30,DLG,Monitoria,Realizada
944,1531,DLG,,NaN,DLG - Santa Casa de Santos,"{'id': 2, 'tipo': 'Monitoria'}",2025-07-28,DLG,Monitoria,Realizada
945,1531,DLG,,NaN,DLG - Santa Casa de Santos,"{'id': 2, 'tipo': 'Monitoria'}",2025-12-08,DLG,Monitoria,Realizada
976,2230,EASi-HF reduzido - 1378-0018,1378-0018,NaN,EASi-HF reduzido - 1378-0018 - Maternidade e C...,"{'id': 2, 'tipo': 'Monitoria'}",2025-10-23,EASi-HF reduzido - 1378-0018,Monitoria,Realizada


In [45]:
def verificar_monitorias_pendentes():
    df = monitorias_filtrado.copy()

    df['data_estimada_filter'] = (
        pd.to_datetime(df['data_estimada_filter'], errors='coerce')
        .dt.tz_localize(None)
    )

    hoje = pd.Timestamp.today().normalize()

    monitorias_pendentes = df[
        (df['ds_descricao'] == 'Pendente') &
        (df['data_estimada_filter'] >= hoje)
    ].sort_values('data_estimada_filter')

    print(
        monitorias_pendentes[
            ['dados_tipo_tipo', 'data_estimada_filter',
             'apelido_protocolo_x', 'ds_descricao', 'apelido_centro']
        ]
    )

    return monitorias_pendentes


    schedule.every().monday.at("09:00").do(verificar_monitorias_pendentes)
monitorias_pendentes = verificar_monitorias_pendentes()


    dados_tipo_tipo data_estimada_filter  apelido_protocolo_x ds_descricao  \
0         Monitoria           2026-02-13           SONATA-HCM     Pendente   
153       Monitoria           2026-02-13      LIBREXIA-STROKE     Pendente   
637       Monitoria           2026-02-13  EASi-HF - 1378-0020     Pendente   
63        Monitoria           2026-02-25              CS45570     Pendente   
635       Monitoria           2026-02-26  EASi-HF - 1378-0020     Pendente   
457       Monitoria           2026-03-09              FREXALT     Pendente   

                                        apelido_centro  
0                      SONATA-HCM - Clínica CardialMed  
153  LIBREXIA-STROKE - Maternidade e Cirurgia Nossa...  
637  EASi-HF - 1378-0020 - Maternidade e Cirurgia N...  
63                              CS45570 - Leforte HMCG  
635  EASi-HF - 1378-0020 - Maternidade e Cirurgia N...  
457                             FREXALT - Leforte HMCG  


In [46]:
# Variavel de envio do email 

envio_para = os.getenv('ENVIO_PARA')

In [47]:
monitorias_pendentes.rename(columns={'apelido_centro':'Estudo/Centro', 'dados_tipo_tipo': 'Monitoria', 'data_estimada_filter': 'Data Estimada','ds_descricao': 'Status'}, inplace=True)

monitorias_pendentes

,id,apelido_protocolo_x,numero_protocolo,co_externo,Estudo/Centro,dados_tipo,Data Estimada,apelido_protocolo_y,Monitoria,Status
0,784,SONATA-HCM,LX4211.1-314-HCM,None,SONATA-HCM - Clínica CardialMed,"{'id': 2, 'tipo': 'Monitoria'}",2026-02-13,SONATA-HCM,Monitoria,Pendente
153,169,LIBREXIA-STROKE,70033093STR3001,STRUCK,LIBREXIA-STROKE - Maternidade e Cirurgia Nossa...,"{'id': 2, 'tipo': 'Monitoria'}",2026-02-13,LIBREXIA-STROKE,Monitoria,Pendente
637,655,EASi-HF - 1378-0020,1378-0020,NaN,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,"{'id': 2, 'tipo': 'Monitoria'}",2026-02-13,EASi-HF - 1378-0020,Monitoria,Pendente
63,2229,CS45570,CS45570,None,CS45570 - Leforte HMCG,"{'id': 2, 'tipo': 'Monitoria'}",2026-02-25,CS45570,Monitoria,Pendente
635,655,EASi-HF - 1378-0020,1378-0020,NaN,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,"{'id': 2, 'tipo': 'Monitoria'}",2026-02-26,EASi-HF - 1378-0020,Monitoria,Pendente
457,584,FREXALT,EFC17919,NaN,FREXALT - Leforte HMCG,"{'id': 2, 'tipo': 'Monitoria'}",2026-03-09,FREXALT,Monitoria,Pendente


In [48]:
monitoria_envio = monitorias_pendentes.drop(columns=['apelido_protocolo_x', 'co_externo', 'dados_tipo', 'apelido_protocolo_y'],errors='ignore')

monitoria_envio = monitoria_envio[
    [
        'Monitoria',
        'id',
        'numero_protocolo',
        'Estudo/Centro',
        'Data Estimada',
        'Status'
    ]
]
monitoria_envio

,Monitoria,id,numero_protocolo,Estudo/Centro,Data Estimada,Status
0,Monitoria,784,LX4211.1-314-HCM,SONATA-HCM - Clínica CardialMed,2026-02-13,Pendente
153,Monitoria,169,70033093STR3001,LIBREXIA-STROKE - Maternidade e Cirurgia Nossa...,2026-02-13,Pendente
637,Monitoria,655,1378-0020,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,2026-02-13,Pendente
63,Monitoria,2229,CS45570,CS45570 - Leforte HMCG,2026-02-25,Pendente
635,Monitoria,655,1378-0020,EASi-HF - 1378-0020 - Maternidade e Cirurgia N...,2026-02-26,Pendente
457,Monitoria,584,EFC17919,FREXALT - Leforte HMCG,2026-03-09,Pendente


In [49]:
def enviar_emails(monitoria_envio):
    global envio_para
    try:
        if monitoria_envio is None or monitoria_envio.empty:
            print("Não há visitas nos próximos dias para enviar por e-mail.")
            return

        tabela_html = monitoria_envio[['Estudo/Centro', 'Monitoria', 'Data Estimada', 'Status']].to_html(
            index=False, escape=False, justify="left", border=0, classes="table"
        )

        msg = MIMEMultipart("alternative")
        msg['From'] = email_usuario
        msg['To'] = email_usuario
        msg['Subject'] = "Visitas de Monitoria Pendentes"

        # Validação da lista de destinatários
        if isinstance(envio_para, list):
            # Expressão regex para validar e-mails
            email_regex = r'^[\w\.-]+@[\w\.-]+\.\w+$'
            envio_para = [email.strip() for email in envio_para if email.strip() and re.match(email_regex, email.strip())]
        else:
            envio_para = []

        # Verifica se há destinatários antes de enviar
        destinatarios = [email_usuario] + envio_para
        if not destinatarios:
            print("Nenhum destinatário válido encontrado. O e-mail não foi enviado.")
            return

        # Somente adiciona Bcc se houver destinatários válidos
        if envio_para:
            msg['Bcc'] = ', '.join(envio_para)

        body = f"""
        <html>
            <head>{css_hover}</head>
            <body>
                <p>Olá,</p>
                <p>Segue abaixo lista com visitas de monitoria programadas para os pr&oacute;ximos dias.</p>
                {tabela_html}
                <p>Este email é gerado automaticamente a partir de informações inseridas na Polo Trial.</p>
                <p>Qualquer dúvida, por favor, contate o <strong><span style="text-decoration: underline;">time BI - SVRI</span></strong>.</p>
            </body>
        </html>
        """
        msg.attach(MIMEText(body, 'html'))

        with smtplib.SMTP(smtp_server, email_port) as server:
            server.starttls()
            server.login(email_usuario, email_senha)
            # Envio corrigido para passar apenas destinatários válidos
            server.sendmail(email_usuario, destinatarios, msg.as_string())

        print("E-mail de visitas pendentes enviado com sucesso!")

    except Exception as e:
        print(f"Erro ao enviar o e-mail: {e}")

# Chamada da função
enviar_emails(monitoria_envio)

E-mail de visitas pendentes enviado com sucesso!
